<a href="https://colab.research.google.com/github/KyoungmiKwon/Bigdata_Training_at_ITwill/blob/main/Team_Project/B01_SJ_news_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os

#저장 경로 지정

os.chdir('/content/drive/MyDrive/삼성전자')

#===============================================================================================================================
# 코드 참조, company_list 다운
# https://blog.naver.com/hengju2/222132075354
# 종목번호 {'삼성전자':'005930', 'LG전자':'066570', 'JYP Ent.':'035900', '와이지엔터테인먼트':'122870'}
#===============================================================================================================================

def crawler(company,company_code,startpage, maxpage):
    result_x={"date" : [], "source" :[], "title" : [],"url" :[], 'article': []}
    data_x=pd.DataFrame(result_x)

    page = int(startpage)

    while page <= int(maxpage):
        url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page)
        source_code = requests.get(url).text
        html = BeautifulSoup(source_code, "lxml")

        # 뉴스 제목
        titles = html.select('.title')

        title_result=[]
        for title in titles:
            title = title.get_text()
            title = re.sub('\n','',title)
            title_result.append(title)

        # 뉴스 링크, 기사
        links = html.select('.title')

        link_result =[]
        art_mainn = []
        try:
            for link in links:
                add = 'https://finance.naver.com' + link.find('a')['href']
                response = requests.get(add).text
                htmlx = BeautifulSoup(response, "lxml")
                selector='#news_read'
                art_main1 = htmlx.select(selector)
                for tag in art_main1[0].find_all('a'):  
                    tag.string.replace_with('')
                art_main = art_main1[0].text.replace('관련뉴스해당 언론사에서 선정하며 언론사 페이지(아웃링크)로 이동해 볼 수 있습니다.','').strip()
                art_mainn.append(art_main)

                link_result.append(add)
        except:
            pass


        # 뉴스 날짜
        dates = html.select('.date')
        date_result = [date.get_text() for date in dates]

        # 뉴스 매체
        sources = html.select('.info')
        source_result = [source.get_text() for source in sources]

        # 다운로드
        
        result= {"date" : date_result, "source" : source_result, "title" : title_result,"url" : link_result, 'article': art_mainn}
        df_result = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in result.items()])) 
        data_x=pd.concat([data_x,df_result],ignore_index=True)
        print("다운 받고 있습니다------")
        page += 1

    data_x.drop_duplicates(["title"],keep = 'last')
    data_x.to_csv(company + company_code + '.csv', mode='w', encoding='utf-8-sig')


        




# 종목 리스트 파일 열기
# 회사명을 종목코드로 변환
def convert_to_code(company,startpage, maxpage):
    file_path='/content/drive/MyDrive/company_list.txt'
    data = pd.read_csv(file_path, dtype=str, sep='\t') # 종목코드 추출
    company_name = data['회사명']
    keys = [i for i in company_name] #데이터프레임에서 리스트로 바꾸기
    company_code = data['종목코드']

    values = [j for j in company_code]
    dict_result = dict(zip(keys, values)) # 딕셔너리 형태로 회사이름과 종목코드 묶기
    pattern = '[a-zA-Z가-힣]+'

    if bool(re.match(pattern, company)) == True: # Input에 이름으로 넣었을 때
        company_code = dict_result.get(str(company))
        crawler(company,company_code,startpage ,maxpage)

    else: # Input에 종목코드로 넣었을 때
        company_code = str(company)
        crawler(company,company_code,startpage, maxpage)




def main():
    info_main = input("="*50+"\n"+"실시간 뉴스기사 다운받기."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    company = input("종목 이름이나 코드 입력: ")
    startpage = input("뉴스 시작 페이지 입력: ")
    maxpage = input("뉴스 마지막 페이지 입력: ")
    convert_to_code(company,startpage, maxpage)


In [3]:
# 코드 참조, company_list 다운
# https://blog.naver.com/hengju2/222132075354

In [8]:
main()

실시간 뉴스기사 다운받기.
 시작하시려면 Enter를 눌러주세요.
==================================================LG전자
종목 이름이나 코드 입력: LG전자
뉴스 시작 페이지 입력: 0
뉴스 마지막 페이지 입력: 5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


다운 받고 있습니다------
다운 받고 있습니다------
다운 받고 있습니다------
다운 받고 있습니다------
다운 받고 있습니다------
다운 받고 있습니다------
